In [10]:
# - command: web IP; open website at IP

In [11]:
import enum
import pathlib
import re
import typing

import pydantic

import utils.computer

In [12]:
NET_PATH = "test.json"

In [13]:
class DeviceType(enum.StrEnum):
    TERMINAL = "terminal"
    CYBERDECK = "cyberdeck"
    SERVER = "server"
    IMPLANT = "implant"

class Manufacturer(enum.StrEnum):
    OUTEL = "Outel"
    BMD = "BMD"
    CYCLOPS = "Cyclops"
    ACRON = "Acron"

class OperatingSystem(enum.StrEnum):
    ORACLEOS = "oracleOS"
    CYCLOPSOS = "CyclopsOS"

In [ ]:
class NPv5Address:
    """Net Protocol v5 address. It is represented as 8 bytes seperated by dots."""
    NUM_BYTES: int = 8

    def __init__(self, address: "str | int | NPv5Address") -> None:
        """Initialize the address."""
        self.address: int = self._parse(address)

    def __repr__(self) -> str:
        """Official string representation."""
        return f"{self.__class__.__name__}({self.address})"

    def __str__(self) -> str:
        """Informal string representation."""
        return ".".join(map("{:02X}".format, list(self.address.to_bytes(self.NUM_BYTES))))

    def _parse(self, address: "str | int | NPv5Address") -> int:
        """Parse address to int from str, int, or address."""
        if isinstance(address, NPv5Address):
            return address.address
        elif isinstance(address, str):
            if match := re.fullmatch(r"\.".join(self.NUM_BYTES * ["[0-9a-fA-F]{1,2}"]), address):
                return int.from_bytes(bytes.fromhex(match.string.replace(".", "")))
        elif isinstance(address, int):
            if 0 <= address <= 2**(self.NUM_BYTES * 8) - 1:
                return address
        raise TypeError("Type not accepted.")

In [15]:
# FIXME: replace computer with device?
class Computer(pydantic.BaseModel):
    """Computer representation."""
    model_config = pydantic.ConfigDict(arbitrary_types_allowed=True)
    net_address: NPv5Address
    ports: list[utils.computer.Ports] = []
    running_programs: list[None] = []  # actually program class? -> websites?
    # file_system: utils.computer.FileSystem = utils.computer.FileSystem()

    @pydantic.field_serializer("net_address")
    def _serialize_address(self, net_address: NPv5Address) -> int:
        """Serialize address (returns address as int)."""
        return net_address.address

    @pydantic.field_validator("net_address", mode="before")
    @classmethod
    def _validate_address(cls, value: int) -> NPv5Address:
        """Validate address (initialises NPv5Address with value)."""
        return NPv5Address(value)

In [16]:
SYSINFO_TEXT = """
OS:         {os}
Kernel:     {kernel}
Uptime:     {uptime}
Shell:      {shell}
CPU:        {cpu}
Memory:     {memory}
"""


class Device(pydantic.BaseModel):
    """Device respresentation."""
    model_config = pydantic.ConfigDict(arbitrary_types_allowed=True)
    net_address: NPv5Address
    # can be terminal, computer, server, brain computer, car, ...
    # do we use an actual type? (for game backend systems only)
    name: str
    """Set by user of the device."""
    type: DeviceType
    """Type of the device."""
    # hardware info
    manufacturer: Manufacturer
    """Name of the manufacturer."""
    device_name: str
    """Name of the device."""

    @pydantic.field_serializer("net_address")
    def _serialize_net_address(self, net_address: NPv5Address) -> int:
        """Serialize net_address (returns address as int)."""
        return net_address.address

    @pydantic.field_validator("net_address", mode="before")
    @classmethod
    def _validate_net_address(cls, value: int) -> NPv5Address:
        """Validate net_address (initialises NPv5Address with value)."""
        return NPv5Address(value)

    def sysinfo(self) -> str:
        return SYSINFO_TEXT.format_map({"os": "oracleOS", "kernel": "lunix", "uptime": "0d 3h 17m 9s",
                                        "shell": "bosh", "cpu": "Cyclops i803", "memory": "???"})

In [ ]:
device = Device(net_address=987654321, name="zer0's Computer", type=DeviceType.TERMINAL,
                manufacturer=Manufacturer.ACRON, device_name="?")

In [18]:
print(device.sysinfo())


OS:         oracleOS
Kernel:     lunix
Uptime:     0d 3h 17m 9s
Shell:      bosh
CPU:        Cyclops i803
Memory:     ???



In [19]:
class Net(pydantic.BaseModel):
    """Net/world representation."""
    home: Computer
    dns: dict[str, str]
    computers: list[Computer]
    network_graph: list[tuple[str, str]]

    def store(self) -> None:
        """Store the config in a file."""
        pathlib.Path(NET_PATH).write_text(
            self.model_dump_json(), encoding="utf-8")